In [1]:
import s3fs
import json
import xarray as xr

In [2]:
with open('aws_creds.json', 'r') as ifile:
    aws_creds = json.loads(ifile.read())

In [14]:
lat, lon = -22.907104, -47.063240

In [60]:
import numpy as np

In [8]:
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False, 
                           config_kwargs = {'max_pool_connections': 50})
    return s3fs.S3Map(path, s3=fs)

In [48]:
testfile = s3open('s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp119/r1i1p1f1/day/tasmax/gr1/v20180701/')

In [79]:
ds = xr.open_mfdataset([testfile], engine='zarr', parallel=True, chunks={'time': 420, 'lat': 180, 'lon': 288})

C:\Users\theodore.wong\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\core\dataset.py:269: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 420. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [57]:
a = ds.tasmax - 273.15

In [64]:
b = a.median(dim='time')

In [66]:
b.values

KeyboardInterrupt: 

In [56]:
ds.tasmax

<xarray.DataArray 'tasmax' (time: 31390, lat: 180, lon: 288)>
dask.array<open_dataset-tasmax, shape=(31390, 180, 288), dtype=float32, chunksize=(420, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: mean time: maximum
    interp_method:  conserve_order2
    long_name:      Daily Maximum Near-Surface Air Temperature
    original_name:  tasmax
    standard_name:  air_temperature
    units:          K

In [71]:
print(ds)

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 31390)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    height     float64 ...
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
Data variables:
    tasmax     (time, lat, lon) float32 dask.array<chunksize=(420, 180, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                <nu

In [80]:
%%time
a = ds['tasmax'].sel(time=slice('2030-01-01', '2049-12-31')).sel(lat=lat, lon=lon, method='nearest').to_numpy()

CPU times: total: 38.2 s
Wall time: 11min 51s


In [70]:
a

array([295.01254, 294.5169 , 294.71484, ..., 295.30386, 295.33215,
       295.33316], dtype=float32)

In [47]:
test

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 7300)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    height     float64 ...
    lat        float64 -22.5
    lat_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lon        float64 0.625
    lon_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
  * time       (time) object 2030-01-01 12:00:00 ... 2049-12-31 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(7300, 2), meta=np.ndarray>
Data variables:
    tasmax     (time) float32 dask.array<chunksize=(405,), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                <null ref>
    ...                     ...
    tracking_id:            hdl:21.14100/3f62f060-9554-4040-a4aa-7ce8dcac8c4b...
    variable_id:            tasmax
    variant_info:           N/A
    variant_label:          r1i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/3f62f060-9554-4040-a4aa-7ce8dcac8c4b...
    version_id:             v20180701

In [44]:
a

<xarray.DataArray (variable: 1, time: 7300)>
dask.array<stack, shape=(1, 7300), dtype=float32, chunksize=(1, 420), chunktype=numpy.ndarray>
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    height     float64 ...
    lat        float64 -22.5
    lat_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lon        float64 0.625
    lon_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
  * time       (time) object 2030-01-01 12:00:00 ... 2049-12-31 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(7300, 2), meta=np.ndarray>
  * variable   (variable) object 'tasmax'
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                <null ref>
    ...                     ...
    tracking_id:            hdl:21.14100/3f62f060-9554-4040-a4aa-7ce8dcac8c4b...
    variable_id:            tasmax
    variant_info:           N/A
    variant_label:          r1i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/3f62f060-9554-4040-a4aa-7ce8dcac8c4b...
    version_id:             v20180701

In [32]:
%%time
b = a.as_numpy()

KeyboardInterrupt: 

In [2]:
s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp119/r1i1p1f1/day/tasmax/gr1/v20180701/

Exception: Missing/incomplete configuration file: C:\Users\theodore.wong/.cdsapirc